### RFM 분석 실습
- Recency: 얼마나 최근에 구매했는가
- Frequency: 얼마나 자주 구매했는가
- Monetary: 얼마나 많은 금액을 지출했는가

### 고객 분석

> voice_no: 송장 번호. 명사 같은. 각 작업에 고유하게 할당된 문자 'I'와 6자리 정수의 조합  
customer_id: 고객번호. 명사 같은. 각 작업에 고유하게 할당된 문자 'C'와 6자리 정수의 조합  
gender: 고객 성별의 문자a열 변수  
age: 고객 연령의 양의 정수 변수  
category: 구매한 상품 카테고리의 문자열 변수 
quantity: 거래당 각 제품(항목)의 수량  
price: 단가. 숫자. 단위당 제품 가격은 터키 리라(TL)  
payment_method: 거래에 사용된 결제 방법(현금, 신용 카드 또는 직불 카드)의 문자열 변수  
invoice_date: 송장 날짜. 거래가 발생한 날  
shopping_mall: 거래가 이루어진 쇼핑몰 이름의 문자열 변수  

### Ⅰ. 데이터 탐색

In [1]:
import pandas as pd

cs_df = pd.read_csv("./datasets/customer_shopping_data.csv")
display(cs_df)

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


#### ✏️ 기획 
1. 쇼핑몰 별 이용 빈도 분석(인기순,,,)
2. 쇼핑몰 카테고리.. 분석 뭐 파는 곳인지? > 그래서 유사한 거 끼리 분석할 수 있을듯
4. 쇼핑목 별 주 사용자 연령층, 성별 분석
5. 쇼핑몰 별..rfm해야될듯 전체에서 하는ㄴ 거 아니고 쇼핑몰 인원끼리 모아서 총합하고 구해야됨!!!(최근 이용 날짜는 평균으로 구할 수 있지 않을까?)
6. 매출 분석

#### 📊 데이터 정보 확인

In [2]:
# cs_df 의 정보 가져오기
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


In [3]:
# 쇼핑몰 확인
cs_df['shopping_mall'].value_counts()

shopping_mall
Mall of Istanbul     19943
Kanyon               19823
Metrocity            15011
Metropol AVM         10161
Istinye Park          9781
Zorlu Center          5075
Cevahir AVM           4991
Forum Istanbul        4947
Viaport Outlet        4914
Emaar Square Mall     4811
Name: count, dtype: int64

In [4]:
# 상품 카테고리 확인
cs_df['category'].value_counts()

category
Clothing           34487
Cosmetics          15097
Food & Beverage    14776
Toys               10087
Shoes              10034
Souvenir            4999
Technology          4996
Books               4981
Name: count, dtype: int64

In [5]:
# 결제 방식 확인
cs_df['payment_method'].value_counts()

payment_method
Cash           44447
Credit Card    34931
Debit Card     20079
Name: count, dtype: int64

#### 📊 결측치 검사

In [6]:
# cs_df의 결측치 개수를 가져오기
cs_df.isna().sum()

invoice_no        0
customer_id       0
gender            0
age               0
category          0
quantity          0
price             0
payment_method    0
invoice_date      0
shopping_mall     0
dtype: int64

#### 📊 중복행 검사

In [7]:
# cs_df의 중복행 개수를 가져오기
cs_df.duplicated().sum()

0

#### 💡 분석
- 쇼핑몰 별 거래 데이터 프레임의 전체 정보
- 10개 컬럼이 존재함.
- 99457개 데이터 존재함.
- 결측치가 존재하지 않음.
- 중복 데이터가 존재하지 않음.

#### ❓ 가설
- 환불된 주문 데이터가 있을까?

#### 📊 숫자형 데이터 개수, 평균, 표준편차, 최소값, 사분위 분포도, 최대값 확인

In [8]:
cs_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,99457.0,43.427089,14.990054,18.00,30.00,43.0,56.00,69.0
quantity,99457.0,3.003429,1.413025,1.00,2.00,3.0,4.00,5.0
price,99457.0,689.256321,941.184567,5.23,45.45,203.3,1200.32,5250.0


#### 💡 분석
- 고객의 평균 연령은 약 43살으로 나타남.
- 최고 연령은 69살이며, 최소 연령은 18으로 나타남.
- 고객 당 평균 약 1.4개의 제품을 구매한 것으로 나타남.
- 상품 단가는 평균 약 689.427로 나타남.
- 상품의 최고 단가는 5250이며, 최소 단가는 5.23으로 나타남.

#### 📌 시사점
- 상품 최소 단가가 음수로 나오지 않았기 때문에 환불된 데이터가 없는 것으로 판단함.
- 나이, 개수는 이상치 제거가 적절하지 않다고 판단되므로, 별도의 이상치 제거는 실시하지 않음.

#### ❓ 가설
- 주문번호 중 I로 시작하지 않는 주문번호가 있을까?
- 고객번호 중 C로 시작하지 않는 고객번호가 있을까?

#### 📊 주문번호 및 고객번호 이상치 확인

In [9]:
# 주문번호 중 앞글자가 I로 시작하지 않는 주문번호 확인
cs_df[~cs_df['invoice_no'].str.startswith('I')]

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall


In [10]:
# 고객번호 중 앞글자가 C로 시작하지 않는 고객번호 확인
cs_df[~cs_df['customer_id'].str.startswith('C')]

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall


#### 💡 분석
- 모든 주문번호는 동일하게 I로 시작함.
- 모든 고객번호는 동일하게 C로 시작함.

#### 📌 시사점
- 쇼핑몰에 따라 주문번호 및 고객번호의 양식이 달라지지 않으며, 모두 동일한 양식을 띄고 있다.

#### ❓ 가설
- 쇼핑몰에 따라 주문번호 및 고객번호가 달라지지 않는다면, 쇼핑몰은 다르지만 주문번호 및 고객번호가 중복되는 경우가 있을까?

In [11]:
# 주문번호 데이터 개수 확인
cs_df['invoice_no'].value_counts()

invoice_no
I138884    1
I291671    1
I175779    1
I226143    1
I683818    1
          ..
I319287    1
I173400    1
I828064    1
I194850    1
I232867    1
Name: count, Length: 99457, dtype: int64

In [12]:
# 고객번호 데이터 개수 확인
cs_df['customer_id'].value_counts()

customer_id
C241288    1
C116138    1
C382765    1
C285074    1
C405356    1
          ..
C220083    1
C286933    1
C301304    1
C214184    1
C273973    1
Name: count, Length: 99457, dtype: int64

#### 💡 분석
- 주문번호와 고객번호는 각 99457개가 존재하며, 중복되는 값이 없음.

### Ⅱ. 데이터 전처리

In [13]:
# 전처리 전 원본 데이터 보존을 위해 복사하여 새로운 데이터 프레임에 담아주기
cs_copy_df = cs_df.copy()
cs_copy_df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.60,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,16/03/2021,Istinye Park


#### 📊 invoice_date 현재 기준으로 변경

In [14]:
# invoice_data 데이터 변환
# 15/10/2022 형식에서 dayfirst=True를 통해 날짜가 먼저 오는 형식을 사용한다는 것을 나타낸다.
cs_copy_df['invoice_date'] = pd.to_datetime(cs_copy_df['invoice_date'], dayfirst=True)

# 날짜 내림차순 정렬
cs_copy_df.sort_values(by=['invoice_date'], ascending=False).reset_index(drop=True)

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I289611,C222906,Female,45,Clothing,3,900.24,Debit Card,2023-03-08,Cevahir AVM
1,I271683,C180068,Male,55,Shoes,1,600.17,Credit Card,2023-03-08,Emaar Square Mall
2,I312534,C235553,Female,68,Food & Beverage,5,26.15,Credit Card,2023-03-08,Emaar Square Mall
3,I345170,C108141,Female,20,Food & Beverage,1,5.23,Credit Card,2023-03-08,Kanyon
4,I254758,C178144,Female,51,Cosmetics,5,203.30,Cash,2023-03-08,Metropol AVM
...,...,...,...,...,...,...,...,...,...,...
99452,I224830,C254068,Female,30,Food & Beverage,3,15.69,Cash,2021-01-01,Metrocity
99453,I211131,C280473,Male,51,Cosmetics,2,81.32,Cash,2021-01-01,Istinye Park
99454,I192911,C642829,Female,66,Food & Beverage,5,26.15,Credit Card,2021-01-01,Metrocity
99455,I474975,C193810,Female,20,Clothing,2,600.16,Credit Card,2021-01-01,Metropol AVM


In [15]:
# 데이터 중 가장 최신 날짜 가져오기
recently_date = pd.to_datetime(cs_copy_df['invoice_date'].max())
print(recently_date)

# 현재 기준으로 변경하기
cs_copy_df['invoice_date'] = (recently_date - pd.to_datetime(cs_copy_df['invoice_date'])).dt.days

# 날짜 오름차순 정렬(최신순)
cs_copy_df.sort_values(by=['invoice_date'], ascending = True).reset_index(drop=True)

2023-03-08 00:00:00


,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I139558,C196970,Male,41,Food & Beverage,4,20.92,Credit Card,0,Metrocity
1,I323613,C276677,Male,41,Clothing,5,1500.40,Credit Card,0,Metropol AVM
2,I498568,C788712,Female,65,Shoes,4,2400.68,Cash,0,Kanyon
3,I122847,C290329,Female,53,Food & Beverage,4,20.92,Credit Card,0,Kanyon
4,I139860,C224033,Male,28,Clothing,5,1500.40,Debit Card,0,Emaar Square Mall
...,...,...,...,...,...,...,...,...,...,...
99452,I371949,C260427,Male,50,Toys,2,71.68,Credit Card,796,Metrocity
99453,I278066,C316581,Female,48,Food & Beverage,2,10.46,Cash,796,Kanyon
99454,I115941,C105769,Male,34,Clothing,1,300.08,Cash,796,Istinye Park
99455,I101088,C824542,Female,28,Toys,5,179.20,Debit Card,796,Kanyon


#### 📊 주문 별 총 결제 금액

In [16]:
# 주문 수량과 상품 단가를 곱하여 주문 별 총 결제 금액 구하기
cs_copy_df['total_price'] = cs_copy_df['quantity'] * cs_copy_df['price']
cs_copy_df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,total_price
0,I138884,C241288,Female,28,Clothing,5,1500.40,Credit Card,215,Kanyon,7502.00
1,I317333,C111565,Male,21,Shoes,3,1800.51,Debit Card,451,Forum Istanbul,5401.53
2,I127801,C266599,Male,20,Clothing,1,300.08,Cash,484,Metrocity,300.08
3,I173702,C988172,Female,66,Shoes,5,3000.85,Credit Card,661,Metropol AVM,15004.25
4,I337046,C189076,Female,53,Books,4,60.60,Cash,500,Kanyon,242.40
...,...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.65,Credit Card,168,Kanyon,293.25
99453,I325143,C569580,Male,27,Food & Beverage,2,10.46,Cash,532,Forum Istanbul,20.92
99454,I824010,C103292,Male,63,Food & Beverage,2,10.46,Debit Card,710,Metrocity,20.92
99455,I702964,C800631,Male,56,Technology,4,4200.00,Cash,722,Istinye Park,16800.00


#### 📊 Recency (쇼핑몰 별)

In [22]:
# 쇼핑몰 별로 구매 일자에 대한 평균을 구하고 새로운 데이터 프레임에 담아주기
rfm_df = cs_copy_df.groupby('shopping_mall')['invoice_date'].mean().reset_index(name='recently_mean_date')
rfm_df

,shopping_mall,recently_mean_date
0,Cevahir AVM,405.023843
1,Emaar Square Mall,395.769071
2,Forum Istanbul,398.580554
3,Istinye Park,395.735610
4,Kanyon,394.678404
5,Mall of Istanbul,395.740912
6,Metrocity,397.105989
7,Metropol AVM,398.891546
8,Viaport Outlet,401.312983
9,Zorlu Center,394.638424


#### 📊 Frequency (쇼핑몰 별)

In [26]:
rfm_df['invoice_count'] = cs_copy_df.groupby('shopping_mall')['invoice_no'].count().reset_index()['invoice_no']
rfm_df

,shopping_mall,recently_mean_date,invoice_count
0,Cevahir AVM,405.023843,4991
1,Emaar Square Mall,395.769071,4811
2,Forum Istanbul,398.580554,4947
3,Istinye Park,395.735610,9781
4,Kanyon,394.678404,19823
5,Mall of Istanbul,395.740912,19943
6,Metrocity,397.105989,15011
7,Metropol AVM,398.891546,10161
8,Viaport Outlet,401.312983,4914
9,Zorlu Center,394.638424,5075


#### 📊 Monetary (쇼핑몰 별)

In [27]:
rfm_df['all_total_price'] = cs_copy_df.groupby('shopping_mall').sum('total_price').reset_index()['total_price']
rfm_df

,shopping_mall,recently_mean_date,invoice_count,all_total_price
0,Cevahir AVM,405.023843,4991,12645138.20
1,Emaar Square Mall,395.769071,4811,12406100.29
2,Forum Istanbul,398.580554,4947,12303921.24
3,Istinye Park,395.735610,9781,24618827.68
4,Kanyon,394.678404,19823,50554231.10
5,Mall of Istanbul,395.740912,19943,50872481.68
6,Metrocity,397.105989,15011,37302787.33
7,Metropol AVM,398.891546,10161,25379913.19
8,Viaport Outlet,401.312983,4914,12521339.72
9,Zorlu Center,394.638424,5075,12901053.82


In [29]:
# 중복값 검사
rfm_df.duplicated().sum()

0

### Ⅲ. 정규화

In [38]:
from sklearn.preprocessing import MinMaxScaler

# 데이터 프레임 복사
rfm = rfm_df.copy()
# shopping_mall 피처 삭제
rfm = rfm.drop(labels=['shopping_mall'], axis=1)

# 정규화 진행
normalization = MinMaxScaler()
rfm_normalization = normalization.fit_transform(rfm)

# 정규화된 데이터를 데이터 프레임으로 변환, 피처는 rfm 데이터 프레임과 동일하게 설정
rfm = pd.DataFrame(rfm_normalization, columns=rfm.columns)

# recently_mean_date는 현재 낮을 수록 최신에 가깝고, 최신 점수가 더 높아야 하기 때문에 1을 빼서 점수를 높여준다.
rfm['recently_mean_date'] = 1 - rfm['recently_mean_date']
rfm

,recently_mean_date,invoice_count,all_total_price
0,0.000000,0.011895,0.008847
1,0.891131,0.000000,0.002649
2,0.620417,0.008988,0.000000
3,0.894353,0.328443,0.319299
4,0.996150,0.992070,0.991748
5,0.893843,1.000000,1.000000
6,0.762401,0.674068,0.648167
7,0.590472,0.353555,0.339032
8,0.357314,0.006807,0.005637
9,1.000000,0.017446,0.015482


#### 📊 쇼핑몰 등급(순위)

In [52]:
# rfm을 더하여 TotalScore 구하기
rfm_df['TotalScore'] = rfm[['recently_mean_date', 'invoice_count', 'all_total_price']].sum(axis=1)
# 순위별로 오름차순 정렬
rfm_df = rfm_df.sort_values(by=['TotalScore'], ascending = False).reset_index(drop=True)
# 정렬 순서대로 순위 넣기
rfm_df['rank'] = rfm_df['TotalScore'].rank(ascending = False)

#현재 순위가 실수로 나오기 때문에, 정수로 타입 변환
rfm_df = rfm_df.astype({'rank': 'int8'})
rfm_df

,shopping_mall,recently_mean_date,invoice_count,all_total_price,TotalScore,rank
0,Metropol AVM,398.891546,10161,25379913.19,2.979969,1
1,Zorlu Center,394.638424,5075,12901053.82,2.893843,2
2,Emaar Square Mall,395.769071,4811,12406100.29,2.084636,3
3,Istinye Park,395.735610,9781,24618827.68,1.542095,4
4,Forum Istanbul,398.580554,4947,12303921.24,1.283060,5
5,Cevahir AVM,405.023843,4991,12645138.20,1.032929,6
6,Mall of Istanbul,395.740912,19943,50872481.68,0.893781,7
7,Metrocity,397.105989,15011,37302787.33,0.629404,8
8,Viaport Outlet,401.312983,4914,12521339.72,0.369758,9
9,Kanyon,394.678404,19823,50554231.10,0.020742,10
